In [57]:
# basic set of imports
import re, operator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler

from IPython.display import display, HTML
import datetime, pickle
import time
import tables
%matplotlib inline

import ml_metrics as metrics
import random

In [5]:
filename = "destinations"
destinations = pd.read_csv(r"..\expedia_data\{0}.csv".format(filename))


In [13]:
filename = 'test_as_hdf'
test = pd.read_hdf(r"..\expedia_data\{0}.h5".format(filename))
test.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2528243 entries, 0 to 2528242
Data columns (total 22 columns):
id                           uint32
date_time                    datetime64[ns]
site_name                    uint8
posa_continent               uint8
user_location_country        uint8
user_location_region         uint16
user_location_city           uint16
orig_destination_distance    float64
user_id                      uint32
is_mobile                    uint8
is_package                   uint8
channel                      uint8
srch_ci                      datetime64[ns]
srch_co                      datetime64[ns]
srch_adults_cnt              uint8
srch_children_cnt            uint8
srch_rm_cnt                  uint8
srch_destination_id          uint32
srch_destination_type_id     uint8
hotel_continent              uint8
hotel_country                uint8
hotel_market                 uint16
dtypes: datetime64[ns](3), float64(1), uint16(3), uint32(3), uint8(12)
memory usag

In [40]:
filename = 'train_as_hdf'
train = pd.read_hdf(r"..\expedia_data\{0}.h5".format(filename))
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37670293 entries, 0 to 37670292
Data columns (total 24 columns):
date_time                    datetime64[ns]
site_name                    uint8
posa_continent               uint8
user_location_country        uint8
user_location_region         uint16
user_location_city           uint16
orig_destination_distance    float64
user_id                      uint32
is_mobile                    uint8
is_package                   uint8
channel                      uint8
srch_ci                      datetime64[ns]
srch_co                      datetime64[ns]
srch_adults_cnt              uint8
srch_children_cnt            uint8
srch_rm_cnt                  uint8
srch_destination_id          uint32
srch_destination_type_id     uint8
is_booking                   uint8
cnt                          uint16
hotel_continent              uint8
hotel_country                uint8
hotel_market                 uint16
hotel_cluster                uint8
dtypes: da

In [55]:
import gc
gc.collect()

0

In [47]:
train["hotel_cluster"].value_counts()


91    1043720
41     772743
48     754033
64     704734
65     670960
5      620194
98     589178
59     570291
42     551605
21     550092
70     545572
18     545284
83     534132
46     534038
25     530591
62     518809
95     509266
28     507016
68     503797
82     503755
37     496061
50     489892
30     489287
9      488328
58     483253
97     479446
16     477868
72     457463
1      452694
99     444887
       ...   
19     282893
84     278264
66     273505
38     269246
87     260398
23     259233
12     259022
31     257587
67     255946
43     253578
7      252447
54     250745
92     244343
89     243560
45     241408
49     240124
3      225250
80     220218
60     217919
71     216054
93     214293
86     209054
14     192299
75     165226
24     164127
35     139122
53     134812
88     107784
27     105040
74      48355
Name: hotel_cluster, dtype: int64

In [41]:
train.shape

(37670293, 24)

In [42]:
test.shape


(2528243, 22)

In [48]:
test_ids = set(test.user_id.unique())

In [49]:
train_ids = set(train.user_id.unique())

In [50]:
intersection_count = len(test_ids & train_ids)
intersection_count == len(test_ids)

True

In [51]:
train["year"] = train["date_time"].dt.year
train["month"] = train["date_time"].dt.month

In [52]:
train.columns

Index(['date_time', 'site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id',
       'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market',
       'hotel_cluster', 'year', 'month'],
      dtype='object')

In [53]:
unique_users = train.user_id.unique()

In [58]:
sel_user_ids = [unique_users[i] for i in sorted(random.sample(range(len(unique_users)), 10000)) ]

In [59]:
sel_train = train[train.user_id.isin(sel_user_ids)]

In [60]:
t1 = sel_train[((sel_train.year == 2013) | ((sel_train.year == 2014) & (sel_train.month < 8)))]


In [61]:
t2 = sel_train[((sel_train.year == 2014) & (sel_train.month >= 8))]
t2 = t2[t2.is_booking == True]

In [62]:
most_common_clusters = list(train.hotel_cluster.value_counts().head().index)

In [63]:
most_common_clusters

[91, 41, 48, 64, 65]

In [64]:
predictions = [most_common_clusters for i in range(t2.shape[0])]

In [65]:
target = [[l] for l in t2["hotel_cluster"]]
metrics.mapk(target, predictions, k=5)

0.064144546431820049

In [66]:
## Vik tutorial-maker gets 0.058020770920711007 here - quite a difference - but we *do* have a random sample of 10,000

In [ ]:
train.corr()["hotel_cluster"]